**(AB)⁻¹ = B⁻¹A⁻¹ in Action: Practical Matrix Inverses with Gauss–Jordan**

**Definition of Matrix Inverse**

Let $A$ be a square matrix of size $n \times n$. An $n \times n$ matrix $X$ is called the inverse of $A$ if it satisfies:
 
  $XA = I = AX$,
            
where $I$ is the $n \times n$ identity matrix. The inverse of $A$ is denoted by $A^{-1}$.

**Theorem 1:**

Existence of Inverse
 
A square matrix has an inverse if and only if it is nonsingular.

**Lemma 1:**
Uniqueness of Inverse

The inverse of a square matrix, if it exists, is unique.
 
  **Proof:** 
  
  Suppose both $X$ and $Y$ satisfy $XA = I = AX$ and $YA = I = AY$.
  
  Then,
  \begin{equation}
        X = X I = X(AY) = (XA)Y = IY = Y.
    \end{equation}
    
 Thus, the inverse is unique. 

**Lemma 2:** 

Inverse of an Inverse

 If $A$ is an invertible matrix, then $A^{-1}$ is also invertible and $(A^{-1})^{-1} = A$.
 
 **Proof:**
 
 Since $A^{-1} A = I = A A^{-1}$, it follows that $A$ is the inverse of $A^{-1}$. 

**Lemma 3:** 

Inverse of a Product
    
If $A$ and $B$ are invertible matrices of the same size, then their product $AB$ is invertible, and


$(AB)^{-1} = B^{-1} A^{-1}$
       
**Proof:**
    
Let 
    $X = B^{-1} A^{-1}$
    
Then,

$$X(AB) = (B^{-1}A^{-1})(AB) = B^{-1}(A^{-1}A)B = B^{-1}(I)B = B^{-1}B = I$$

and,

$$(AB)X = (AB)(B^{-1}A^{-1}) = A(BB^{-1})A^{-1} = A(I)A^{-1} = AA^{-1} = I$$

Thus,
$X$ is both a left and a right inverse for $AB$.


**Generalization:**

Inverse of a Product of Matrices
  
$(A_1 A_2 \dots A_k)^{-1} = A_k^{-1} A_{k-1}^{-1} \dots A_2^{-1} A_1^{-1}$.

 Warning:  In general, $(A+B)^{-1} \neq A^{-1} + B^{-1}$.

**Gauss–Jordan Elimination**

Gauss–Jordan Elimination is a fundamental algorithm for computing the inverse of a nonsingular matrix.

Recipe to find $A^{-1}$:

1- Write the augmented matrix $(A∣I)$

2- Make the first pivot 1.

3- Zero out below and above that pivot.

4- Repeat for the next pivot until the left becomes $I$.

5- The right part is $A^{-1}$.

In [24]:
import numpy as np

def gauss_jordan_inverse(matrix: np.ndarray, *, tol: float = 1e-12) -> np.ndarray:
    """Compute the inverse of a square matrix using Gauss–Jordan elimination.

    Uses partial pivoting and raises ValueError if the matrix is singular (non-invertible).
    """
    if matrix.ndim != 2 or matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Input must be a square matrix")

    n = matrix.shape[0]
    augmented_matrix = np.hstack((matrix.astype(float), np.eye(n, dtype=float)))

    for pivot_index in range(n):
        # Partial pivoting: choose the row with the largest absolute pivot in this column
        column_slice = augmented_matrix[pivot_index:, pivot_index]
        max_row_offset = int(np.argmax(np.abs(column_slice)))
        max_row_index = pivot_index + max_row_offset

        # Swap current row with the best pivot row if needed
        if max_row_index != pivot_index:
            augmented_matrix[[pivot_index, max_row_index]] = augmented_matrix[[max_row_index, pivot_index]]

        pivot_value = augmented_matrix[pivot_index, pivot_index]
        if abs(pivot_value) < tol:
            raise ValueError("Matrix is singular or nearly singular; inverse does not exist.")

        # Normalize the pivot row so the pivot becomes 1
        augmented_matrix[pivot_index] = augmented_matrix[pivot_index] / pivot_value

        # Eliminate all other entries in this column
        for row_index in range(n):
            if row_index == pivot_index:
                continue
            elimination_factor = augmented_matrix[row_index, pivot_index]
            if elimination_factor != 0.0:
                augmented_matrix[row_index] = (
                    augmented_matrix[row_index] - elimination_factor * augmented_matrix[pivot_index]
                )

    inverse_matrix = augmented_matrix[:, n:]
    return inverse_matrix

# --- get matrices from user ---
n = int(input("Enter the size of square matrices (n): "))

print("Enter matrix A row by row (numbers separated by space):")
A = []
for i in range(n):
    row = list(map(float, input(f"Row {i+1}: ").split()))
    A.append(row)
A = np.array(A, dtype=float)

print("Enter matrix B row by row (numbers separated by space):")
B = []
for i in range(n):
    row = list(map(float, input(f"Row {i+1}: ").split()))
    B.append(row)
B = np.array(B, dtype=float)

# --- compute inverses ---
try:
    A_inv = gauss_jordan_inverse(A)
    B_inv = gauss_jordan_inverse(B)
except ValueError as e:
    print("\nOne of the matrices is singular. Details:", e)
    raise SystemExit(1)

print("\nA^-1 =\n", A_inv)
print("\nB^-1 =\n", B_inv)

# --- check (AB)^-1 = B^-1 A^-1 ---
AB = A @ B
try:
    AB_inv = gauss_jordan_inverse(AB)
except ValueError as e:
    print("\nProduct AB is singular; cannot compute (AB)^-1. Details:", e)
    raise SystemExit(1)

B_inv_A_inv = B_inv @ A_inv

print("\n(AB)^-1 =\n", AB_inv)
print("\nB^-1 A^-1 =\n", B_inv_A_inv)

# difference to show equality
difference = AB_inv - B_inv_A_inv
print("\nDifference (should be near zero):\n", difference)
print("\nFrobenius norm of difference:", np.linalg.norm(difference, ord='fro'))


Enter the size of square matrices (n): 2
Enter matrix A row by row (numbers separated by space):
Row 1: 1 2
Row 2: 3 4
Enter matrix B row by row (numbers separated by space):
Row 1: 2 1
Row 2: 1 3

A^-1 =
 [[-2.   1. ]
 [ 1.5 -0.5]]

B^-1 =
 [[ 0.6 -0.2]
 [-0.2  0.4]]

(AB)^-1 =
 [[-1.5  0.7]
 [ 1.  -0.4]]

B^-1 A^-1 =
 [[-1.5  0.7]
 [ 1.  -0.4]]

Difference (should be near zero):
 [[-2.22044605e-16  2.22044605e-16]
 [ 1.11022302e-16 -1.11022302e-16]]

Frobenius norm of difference: 3.510833468576701e-16
